## Make a Pickett Plot in Plotly

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

A Pickett plot is a graphical solution to Archie's equation.  Archie developed his famous equation to calculate, from well log parameters, the water saturation (Sw) of the uninvaded zone in a formation next to a borehole. The Archie equation can be expressed as follows:

$S_w^n = \frac{R_w}{\left(\Phi^m R_t\right)}$


- Sw = water saturation of the uninvaded zone
- n = saturation exponent, which varies from 1.8 to 4.0 but normally is 2.0
- Rw = formation water resistivity at formation temperature
- Φ = porosity
- m = cementation exponent, which varies from 1.7 to 3.0 but normally is 2.0
- Rt = true resistivity of the formation, corrected for invasion, borehole, thin bed, and other effects

In [ ]:
df = pd.read_csv('../data/ideal_input_data.csv', skiprows=[1])

In [ ]:
df.describe()

In [ ]:
phi = [0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.18, 0.18, 0.1, 0.1, 0.1, 0.1, 0.1, 0.08, 0.08, .06]
Sw = [0.14, 0.2, 0.24, 0.28, 1, 0.2, 0.3, 0.37, 0.4, 0.5, 1, 0.28, 1, 0.4, 0.6, 0.7, 0.8, 1, 0.5, 1, 1]
R = [1.4, 9, 15, 18,22, 48, 3, 10, 18, 20, 28, 53, 5.5, 60, 13, 20, 25, 32, 64, 20, 30]

df = pd.DataFrame({'phi': phi, 'Sw': Sw, 'R': R})

In [ ]:
fig=px.scatter(df, x="RT", y='PHIX', log_x=True, log_y=True)
fig.show()

Bulk Volume Water: $BVW = \phi * S_w$

BVWirr = 0.04

In [ ]:
R_test = np.linspace(0.001, 1000, 100)
Rw = 0.1 #this can be a parameter

Calculate the constant Sw lines.

In [ ]:
m = 5
b = 1./m * np.log(Rw)
phi_sw_exp = -(1./m)*np.log(R_test) + b
phi_sw = 10**phi_sw_exp

phi_int_exp = b
phi_int = 10**phi_int_exp

b_2 = phi_int_exp + (1./m)*np.log(2)
phi_2_exp = -(1./m)*np.log(R_test) + b_2
phi_2 = 10**phi_2_exp

b_4 = phi_int_exp + (1./m)*np.log(4)
phi_4_exp = -(1./m)*np.log(R_test) + b_4
phi_4 = 10**phi_4_exp

b_6 = phi_int_exp + (1./m)*np.log(6)
phi_6_exp = -(1./m)*np.log(R_test) + b_6
phi_6 = 10**phi_6_exp

b_8 = phi_int_exp + (1./m)*np.log(8)
phi_8_exp = -(1./m)*np.log(R_test) + b_8
phi_8 = 10**phi_8_exp

b_14 = phi_int_exp + (1./m)*np.log(14)
phi_14_exp = -(1./m)*np.log(R_test) + b_14
phi_14 = 10**phi_14_exp

b_20 = phi_int_exp + (1./m)*np.log(20)
phi_20_exp = -(1./m)*np.log(R_test) + b_20
phi_20 = 10**phi_20_exp

## Saturation Lines

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['RT'], y=df['PHIX'],
                         mode='markers',
                         name='Data'))
fig.add_trace(go.Scatter(x=[Rw], y=[1],
                         name='Rw',
                         mode='markers'))
fig.add_trace(go.Scatter(x=R_test, y=phi_sw, name='Sw = 100%'))
fig.add_trace(go.Scatter(x=[1], y=[phi_int],
                         name='phi@R=1',
                         mode='markers'))
fig.add_trace(go.Scatter(x=R_test, y=phi_2, name='Sw = 71%'))
fig.add_trace(go.Scatter(x=R_test, y=phi_4, name='Sw = 50%'))
fig.add_trace(go.Scatter(x=R_test, y=phi_6, name='Sw = 41%'))
fig.add_trace(go.Scatter(x=R_test, y=phi_8, name='Sw = 35%'))
fig.add_trace(go.Scatter(x=R_test, y=phi_14, name='Sw = 27%'))
fig.add_trace(go.Scatter(x=R_test, y=phi_20, name='Sw = 22%'))

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(range=[np.log10(0.01), 3],
                 constrain="domain",
                 type='log',
                 title='Resistivity')

fig.update_yaxes(range=[np.log10(0.01), 0],
                 constrain="domain",
                 type='log',
                 title='$\phi$')
#fig.update_plot(title='Pickett Plot')

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data


# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("petroDash - Pickett"),
    dcc.Graph(id='graph'),
    html.Label([
        "m",
        dcc.Slider(
            id='m-slider',
            min=1,
            max=7,
            step=0.1,
            value=5,
            tooltip={'always_visible':True},
            marks={'1': '1',
                   '7': '7'}
        ),
    ]),
    html.Label([
        "Rw",
        dcc.Slider(
            id='Rw-slider',
            min=0.01,
            max=10,
            step=0.01,
            value=.1,
            tooltip={'always_visible':True},
            marks={'0.01': '0.01',
                   '0.1': '0.1',
                   '1': '1.0',
                   '10': '10'}
        ),
    ]),
    
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("Rw-slider", "value"),
     Input("m-slider", "value")]
)
def update_figure(Rw, m):
    
    R_test = np.linspace(0.001, 1000, 100)
    
    #m = 5
    b = 1./m * np.log(Rw)
    phi_sw_exp = -(1./m)*np.log(R_test) + b
    phi_sw = 10**phi_sw_exp
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['RT'], y=df['PHIX'],
                             mode='markers',
                             name='Data'))
    fig.add_trace(go.Scatter(x=R_test, y=phi_sw, name='Sw = 100%'))
    
    
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )
    fig.update_xaxes(range=[np.log10(0.01), 3],
                     constrain="domain",
                     type='log',
                     title='Resistivity')

    fig.update_yaxes(range=[np.log10(0.01), 0],
                     constrain="domain",
                     type='log',
                     title='$/phi$')
    
    return fig

# Run app and display result inline in the notebook

#app.run_server(mode='external')
app.run_server(mode='inline')
#app.run_server(debug='False',port=8080,host='0.0.0.0')